In [22]:
# Import the CARLA Python API library and some utils
import carla 
import math 
import random 
import time 

client = carla.Client('ines-gpu-01.informatik.uni-mannheim.de', 2001)
client.load_world("Town04")
world = client.get_world()
map = world.get_map()
topology = map.get_topology()
topology = [x[0] for x in topology]
topology = sorted(topology, key=lambda w: w.transform.location.z)


In [29]:
def get_waypoints(waypoint):
    waypoints = [waypoint]
    nxt = waypoint.next(0.05)
    if len(nxt) > 0:
        nxt = nxt[0]
        while nxt.road_id == waypoint.road_id:
            if  nxt.transform.rotation.yaw != waypoints[len(waypoints) - 1].transform.rotation.yaw:
                waypoints.append(nxt)
            else:
                latest = nxt
            nxt = nxt.next(0.05)
            if len(nxt) > 0:
                nxt = nxt[0]
            else:
                break
        if len(waypoints) == 1 and latest:
            waypoints.append(latest)
    return waypoints

In [30]:
set_waypoints = []
for waypoint in topology:
    set_waypoints.append(get_waypoints(waypoint))

In [31]:
from json import JSONEncoder
class WaypointEncoder(JSONEncoder):
    def default(self, waypoint):
        return {
            'x': float('%.3f' % (waypoint.transform.location.x)),
            'y': float('%.3f' % (waypoint.transform.location.y)),
            'rotation': float('%.3f' % (waypoint.transform.rotation.yaw)),
            'is_junction': waypoint.is_junction,
            'lane_width': waypoint.lane_width,
        }

In [32]:
import json
with open("output.json", "w") as f:
    json.dump(set_waypoints, f, cls=WaypointEncoder)